# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
import time
from scipy.interpolate import interp1d

In [ ]:
starry.config.lazy = False
starry.config.quiet = True
np.random.seed(0)

In [ ]:
# Load the data
data = np.load("lightcurve.npz")
time = data["time"]
flux0 = data["flux0"]
ydeg = data["ydeg"]
ydeg_max = data["ydeg_max"]
inc = data["inc"]
y = data["y"]
prot = data["prot"]

In [ ]:
ferr = 1e-4
flux = flux0 + ferr * np.random.randn(len(flux0))

In [ ]:
plt.plot(time, flux, "k.", ms=3, alpha=0.5)
plt.plot(time, flux0, "C0-", lw=1);

In [ ]:
# Get the light curve for each ~period
theta = (360.0 / prot * (time - time[0])) % 360.0
breaks = np.where(np.diff(theta) < 0)[0]
sz = np.max(np.diff(breaks)) + 1
Theta = []
Time = []
Flux = []
i = 0
for b in breaks:
    eps = sz - (b - i)
    idx = slice(i, b + eps)
    t = np.array(theta[idx])
    if t[0] > 180:
        t[0] -= 360.0
    if t[-1] < 180:
        t[-1] += 360.0
    Theta.append(t)
    Time.append(time[idx])
    Flux.append(flux[idx])
    i = b
Theta = np.array(Theta)
Time = np.array(Time)
Flux = np.array(Flux)

In [ ]:
alpha = np.linspace(0, 1.0, 100)
lnlike = np.zeros((len(alpha), len(Theta)))

map = starry.Map(ydeg_max, inc=inc)
for k in tqdm(range(len(alpha))):
    map.alpha = alpha[k]
    for i in range(len(Theta)):
        map.set_data(Flux[i], C=ferr ** 2)
        map.set_prior(L=1e-6)
        lnlike[k, i] = map.lnlike(theta=Theta[i])

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = [
    plt.subplot2grid((4, 1), (0, 0), rowspan=3),
    plt.subplot2grid((4, 1), (3, 0)),
]
ax[0].imshow(
    np.exp(lnlike - lnlike.max(axis=0)).T,
    origin="lower",
    extent=(0, 1.0, 0, len(Theta)),
    aspect="auto",
)
ax[0].axis("off")
ax[1].plot(alpha, np.exp(lnlike.sum(axis=1) - lnlike.sum(axis=1).max()))
ax[1].set_yticks([])
ax[1].set_xlim(0, 1.0)
ax[1].set_xlabel("alpha")
ax[1].axvline(0.025, color="C1");